# Transcription using OpenAi whisper. 

In [1]:
import numpy as np
from collections import Counter
import librosa
import numpy as np
import IPython.display
import IPython
from torchaudio.utils import download_asset
from IPython.display import Audio
import soundfile as sf
import csv
import os
import pandas as pd
import glob
pd.set_option('display.max_rows', None)


In [2]:
pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 11.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 18.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 44.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796910 sha256=2febd70864d72b5d1ec22d389fde82d86c1e754c9dffefbcba9f4cb4f4ad95a7
  Stored in directory: /root/.cache/pip/wheels/b2/13/5f/fe8245f6dc59df505879da4b21299

In [4]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-1679s2xr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-1679s2xr
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.1 MB/s eta 0:00:0000:010:01
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.3.1
    Uninstalling tiktoken-0.3.1:
      Successfully uninstalled tiktoken-0.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai-whisper 20230314 requires tiktoken==0.3.1, but you have tiktoken 0.3.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 6.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 60.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 17.7 MB/s eta 0:00:00
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14759 sha256=f4b39f197b1784fd303b069da936897092e

In [6]:
import whisper
model = whisper.load_model("base.en")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."

)

/opt/conda/lib/python3.10/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):
100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 65.3MiB/s]


Model is English-only and has 71,825,408 parameters.


the output of transcription of whisper asr model, is a dictionary of **text**, **segments**,**language**

**segments** : refers to the segments or portions of the audio that have been transcribed. Each segment typically represents a specific section of the input audio along with its corresponding transcribed text
**the structure of each segment** : includes attributes,
1. id: The ID assigned to the segment. It provides a unique identifier for each segment in the transcription output.
2. seek: The seek value represents the position of the segment within the audio file. It specifies the starting point of the segment, typically in seconds.
3. start: The start time of the segment, indicating the beginning timestamp of the transcribed portion within the audio file.
4. end: The end time of the segment, representing the timestamp when the transcribed portion ends within the audio file.
5. text: The transcribed text for the corresponding segment. It contains the recognized speech for that specific portion of the audio.
6. **tokens:** The tokens attribute represents the transcribed text segmented **into** individual units, often **words**. It provides a list of these tokens for the given segment, Each of these units is then assigned a unique identifier or index. These identifiers allow for efficient representation and processing of the text during model training and inference, These numbers serve as references to specific words or subword units within the transcribed text
7. **temperature**: The temperature value indicates the level of **randomness** or **diversity** in the model's generation process. A higher temperature generally results in more diverse output, while a lower temperature yields more focused and deterministic output.
8. **avg_logprob**: The average log probability of the segment indicates **the model's confidence or certainty for the transcribed text in that particular segment**. Higher values typically indicate higher confidence.
9. **compression_ratio**: The compression ratio represents the ratio of the original audio duration to the duration of the transcribed segment. It provides information about the **efficiency of the model's transcription**, where a higher value suggests more compression.
10. **no_speech_prob**: The probability of no speech occurring in the segment. It represents the **likelihood of there being no speech or silence** within the transcribed portion.



In [7]:
#Calculate lexical diversity, TTR
def calculate_ttr(transcription):
    words = transcription.split()
    #print (words)
    num_words = len(words)
    unique_words = len(set(words))
    ttr = unique_words / num_words
    return ttr

In [8]:
#Simpson's Diversity Index (SDI) for lexical diversity which considers both the number of unique words and their frequency of occurrence.
def calculate_sdi(transcription):
    words = transcription.split()
    word_counts = Counter(words)
    #print(word_counts)
    num_words = len(words)
    sdi = 1 - sum((count / num_words) ** 2 for count in word_counts.values())
    return sdi


In [9]:
#pauses 
def detect_pauses(audio_file, sr, min_pause_duration=0.5):
    # Load audio file
    audio, sr = librosa.load(audio_file, sr=sr)
    
    # Convert audio to mono if it has multiple channels
    if len(audio.shape) > 1:
        audio = np.mean(audio, axis=1)
    
    # Apply voice activity detection (VAD) or other methods to segment speech
    
    # Example using librosa VAD
    speech_segments = librosa.effects.split(audio, top_db=20)
    
    # Calculate pause durations between speech segments
    #sr = bundle.sample_rate
    pause_durations = []
    pauses = []
    prev_end = 0.0
    for start, end in speech_segments:
        pause_duration = start - prev_end
        if pause_duration >= min_pause_duration:
            pauses.append(pause_duration)
        prev_end = end
    for pause in pauses:
        pause_duration = pause / sr
        pause_durations.append(pause_duration)
        
    total_duration = len(audio_file) / sr
    pause_rate = sum(pause_durations) / total_duration
    return pause_rate 


In [10]:
from flair.models import SequenceTagger
from flair.data import Sentence

def calculate_syntactic_complexity(transcription):
    sentences = transcription.split('.')  # Split text into sentences assuming period (.) as the sentence delimiter
    num_sentences = len(sentences)
    
    total_clauses = 0
    num_words = 0
    for sentence in sentences:
        words = sentence.split()
        num_words += len(words)
        
        tagger = SequenceTagger.load('pos')
        line = Sentence(sentence)
        tagger.predict(line)
        pos_tags = [(token.text, token.labels[0].value) for token in line]

        num_clauses = 0
        for i in range(len(pos_tags)-1):
            current_tag = pos_tags[i][1]
            next_tag = pos_tags[i+1][1]
            if current_tag.startswith('V') and next_tag not in ['.', ':', 'IN']:
                num_clauses += 1
        total_clauses += num_clauses
    
    average_sentence_length = num_words / num_sentences
    average_clauses_per_sentence = total_clauses / num_sentences

#     print(num_words)
#     print(num_sentences)
#     print(total_clauses)
    
    
    return average_sentence_length, average_clauses_per_sentence


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
# # Create and open the CSV file for writing
# audio_directories = ["/kaggle/input/adresso-audio-data/AD", "/kaggle/input/adresso-audio-data/CN", "/kaggle/input/adresso-audio-data/MCI"]

#  # Iterate through audio directories
# for directory, label in zip(audio_directories, ["AD", "CN", "MCI"]):
# # Iterate through audio files in each directory
#     output_file = f"output_{label}.csv"
#      # Open the CSV file for writing
#     with open(output_file, 'w', newline='') as csvfile:
#         csv_writer = csv.writer(csvfile)
#         csv_writer.writerow(["Filename", "Transcription", "Label"])
#         #iterate through the audio files in each directory
#         for m in os.listdir(directory):
#             audio_path = os.path.join(directory, m)
#             # Display Audio
#             audio_display = Audio(audio_path)
#             IPython.display.display(audio_display) 
#             # Perform Trancription
#             transcription_dict = model.transcribe(audio_path)
#             transcription_text = transcription_dict['text']
    
#             # Write the filename and transcription to the CSV file
#             csv_writer.writerow([m, transcription_text, label])
    
#             #get Segements and Number of words
#             segments = transcription_dict["segments"]
#             num_words = len(transcription_text.split())
#             print(f"Transcription for {m}: {transcription_text}\n")
    
#             #Calculate speech rate
    
#             audio_info = sf.info(audio_path)
#             duration = audio_info.duration
#             sample_rate = audio_info.samplerate
#             frame_duration = 0.01
#             num_frames = int(duration * sample_rate / frame_duration)
    
#             #The formula calculates the average number of words spoken per second 
#             #based on the number of words in the transcription and the duration of the audio file.
#             speech_rate = num_words / (num_frames / sample_rate)
#             print("Speech Rate (WPS):", speech_rate)
    
    
#             #Lexical diversity using TTR 
#             ttr_score = calculate_ttr(transcription_text)
#             print("TTR Score:", ttr_score)
    
#             #Lexical diversity using sdi 
#             sdi_score = calculate_sdi(transcription_text)
#             print("SDI Score:", sdi_score)
    
#             #Pause Rate 
#             pauses = detect_pauses(audio_path,sr=sample_rate)
#             print("Pause durations:", pauses)
    
#             #Syntactic Complexity:
#             avg_sentence_length, avg_clauses_per_sentence = calculate_syntactic_complexity(transcription_text)
#             print("Average Sentence Length:", avg_sentence_length)
#             print("Average Clauses per Sentence:", avg_clauses_per_sentence)

# Adding new data Pitt

# Performing Transcription and adding 4AD metrics on a data Folder 

In [11]:
# Set the folder path containing the audio files
audio_folder = "/kaggle/input/audio-added-data/Audio_pitt_AD"

# Set the label for all audio files in the folder
label = "AD"

# Set the output CSV file
output_file = f"output_{label}.csv"

# Open the CSV file for writing
with open(output_file, 'a', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Filename", "Transcription", "Label", "Speech Rate", "Pause Rate", "Lexical Diversity(SDI Score)", "Syntactic Complexity(Avg Clauses per)"])

    # Get the list of audio files in the folder
    audio_files = glob.glob(os.path.join(audio_folder, "*.wav")) + glob.glob(os.path.join(audio_folder, "*.mp3"))

    # Loop through each audio file
    for audio_file in audio_files:
        # Display Audio
        audio_display = Audio(audio_file)
        # IPython.display.display(audio_display)
        
        # Perform Transcription
        transcription_dict = model.transcribe(audio_file)
        transcription_text = transcription_dict['text']
    
    #perform 4 features of alzheimers
    #get Segements and Number of words
        segments = transcription_dict["segments"]
        num_words = len(transcription_text.split())
    #print(f"Transcription for {m}: {transcription_text}\n")
    
    #Calculate speech rate
    
        audio_info = sf.info(audio_file)
        duration = audio_info.duration
        sample_rate = audio_info.samplerate
        frame_duration = 0.01
        num_frames = int(duration * sample_rate / frame_duration)
    
    #The formula calculates the average number of words spoken per second 
    #based on the number of words in the transcription and the duration of the audio file.
        speech_rate = num_words / (num_frames / sample_rate)
        print("Speech Rate (WPS):", speech_rate)
    
    
    #Lexical diversity using TTR 
        ttr_score = calculate_ttr(transcription_text)
        print("TTR Score:", ttr_score)
    
    #Lexical diversity using sdi 
        sdi_score = calculate_sdi(transcription_text)
        print("SDI Score:", sdi_score)
    
    #Pause Rate 
        pauses = detect_pauses(audio_file,sr=sample_rate)
        print("Pause:", pauses)
    
    #Syntactic Complexity:
        avg_sentence_length, avg_clauses_per_sentence = calculate_syntactic_complexity(transcription_text)
        print("Average Sentence Length:", avg_sentence_length)
        print("Average Clauses per Sentence:", avg_clauses_per_sentence)
        

        # Write the filename, transcription, and features to the CSV file
        csv_writer.writerow([os.path.basename(audio_file), transcription_text, label, speech_rate, pauses, sdi_score, avg_clauses_per_sentence])


# Print completion message
print("Processing complete.")

/opt/conda/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Speech Rate (WPS): 0.012151795740601932
TTR Score: 0.797752808988764
SDI Score: 0.9819467238984977
Pause: 35669.33333333332


2023-06-16 09:00:52,263 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:00:54,337 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:00:56,224 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:00:58,844 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, 

/opt/conda/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Speech Rate (WPS): 0.013742103087914034
TTR Score: 0.6491228070175439
SDI Score: 0.96398891966759
Pause: 17664.000000000004
2023-06-16 09:01:19,002 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
Average Sentence Length: 57.0
Average Clauses per Sentence: 15.0


...

Speech Rate (WPS): 0.014313764058894358
TTR Score: 0.8433734939759037
SDI Score: 0.9821454492669472
Pause: 47018.666666666664
2023-06-16 09:57:59,593 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:58:00,793 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:58:02,385 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LR

/opt/conda/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Speech Rate (WPS): 0.009182746751280987
TTR Score: 0.7692307692307693
SDI Score: 0.9794748520710059
Pause: 20224.0
2023-06-16 09:58:33,442 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:58:34,524 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:58:35,580 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, 

/opt/conda/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Speech Rate (WPS): 0.01692097682784472
TTR Score: 0.6380368098159509
SDI Score: 0.9811434378410929
Pause: 48440.88888888887
2023-06-16 09:59:21,734 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:59:23,126 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
2023-06-16 09:59:24,181 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-

# Performing transcription and 4 AD metrics on a single file

In [ ]:
audio_file = "D:\\GraduationProject#2\\OpenAi\\Audio_pitt_AD\\audioaudio.MP3"
label = "AD"
output_file = f"output_{label}.csv"

# Open the CSV file for writing
with open(output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Filename", "Transcription", "Label", "Speech Rate", "Pause Rate", "Lexical Diversity(SDI Score)", "Syntactic Complexity(Avg Clauses per)"])

    # Display Audio
    audio_display = Audio(audio_file)
    IPython.display.display(audio_display)

    # Perform Transcription
    transcription_dict = model.transcribe(audio_file)
    transcription_text = transcription_dict['text']
    print(transcription_text)

#perform 4 features of alzheimers
    #get Segements and Number of words
    segments = transcription_dict["segments"]
    num_words = len(transcription_text.split())
    #print(f"Transcription for {m}: {transcription_text}\n")
    
    #Calculate speech rate
    
    audio_info = sf.info(audio_file)
    duration = audio_info.duration
    sample_rate = audio_info.samplerate
    frame_duration = 0.01
    num_frames = int(duration * sample_rate / frame_duration)
    
    #The formula calculates the average number of words spoken per second 
    #based on the number of words in the transcription and the duration of the audio file.
    speech_rate = num_words / (num_frames / sample_rate)
    print("Speech Rate (WPS):", speech_rate)
    
    
    #Lexical diversity using TTR 
    ttr_score = calculate_ttr(transcription_text)
    print("TTR Score:", ttr_score)
    
    #Lexical diversity using sdi 
    sdi_score = calculate_sdi(transcription_text)
    print("SDI Score:", sdi_score)
    
    #Pause Rate 
    pauses = detect_pauses(audio_file,sr=sample_rate)
    print("Pause:", pauses)
    
    #Syntactic Complexity:
    avg_sentence_length, avg_clauses_per_sentence = calculate_syntactic_complexity(transcription_text)
    print("Average Sentence Length:", avg_sentence_length)
    print("Average Clauses per Sentence:", avg_clauses_per_sentence)
        

        # Write the filename, transcription, and features to the CSV file
    csv_writer.writerow([os.path.basename(audio_file), transcription_text, label, speech_rate, pauses, sdi_score, avg_clauses_per_sentence])


# Print completion message
print("Processing complete.")
